In [4]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.optim import AdamW
from transformers import RobertaForSequenceClassification
from sklearn.metrics import f1_score
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader

from torch.optim import AdamW
from transformers import RobertaForSequenceClassification, RobertaConfig
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 
import os 
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch
import utils
from torch.utils.data import Subset, DataLoader


In [1]:
import torch
torch.cuda.is_available()

True

In [3]:
#Se leen los datos y se seleccionan las variables que presentan una diferencia
# entre los individuos que presentan estres y los que no
data_path = Path(os.getcwd()).parent / "data"
silver_path = data_path / "silver" / "dreadditCleanTrain.csv"
df = pd.read_csv(silver_path, usecols= ["text","clean_text","clean_text_sentence_sep",
                                        "singular_pronouns", "avg_word_len",'lex_diversity',"label"])

In [5]:
# Load the configuration for the model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
config = RobertaConfig.from_pretrained(model_name, num_labels=2)  # Set to 2 for binary classification
model = RobertaForSequenceClassification.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

config.id2label = {0: "no estres", 1: "estres"}
config.label2id = {"no estres": 0, "estres": 1}

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

c:\Users\DELLPHOTO\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Be

In [6]:
# Ejemplo de cómo usar el dataset
dataset = SentimentDataset(df['text'].tolist(), df['label'].tolist(), tokenizer, max_length=128)
loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [7]:
# Suponiendo que 'model' y 'loader' están definidos y configurados correctamente
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

# Listas para almacenar las métricas
losses = []
f1_scores = []

In [8]:

# Inicializar el KFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Convertir labels a numpy para el uso con StratifiedKFold
labels = np.array(df['label'].tolist())

# Ejecutar la validación cruzada
f1_scores = []
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    train_subsampler = Subset(dataset, train_idx)
    val_subsampler = Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_subsampler, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_subsampler, batch_size=16, shuffle=False)

    print(f"Training fold {fold+1}")
    fold_f1 = utils.train_and_evaluate(model, train_loader, val_loader, device)
    f1_scores.append(fold_f1)

print(f"Average F1 across folds: {np.mean(f1_scores):.4f}")

Training fold 1
Epoch 1/5, Train Loss: 0.4480, Validation F1: 0.8322
Epoch 2/5, Train Loss: 0.3387, Validation F1: 0.8419
Epoch 3/5, Train Loss: 0.2443, Validation F1: 0.8303
Epoch 4/5, Train Loss: 0.1771, Validation F1: 0.8361
Epoch 5/5, Train Loss: 0.0938, Validation F1: 0.8455
Training fold 2
Epoch 1/5, Train Loss: 0.2930, Validation F1: 0.9095
Epoch 2/5, Train Loss: 0.1883, Validation F1: 0.9866
Epoch 3/5, Train Loss: 0.1107, Validation F1: 0.9809
Epoch 4/5, Train Loss: 0.0670, Validation F1: 0.9811
Epoch 5/5, Train Loss: 0.0597, Validation F1: 0.9675
Training fold 3
Epoch 1/5, Train Loss: 0.0797, Validation F1: 1.0000
Epoch 2/5, Train Loss: 0.0432, Validation F1: 1.0000
Epoch 3/5, Train Loss: 0.0299, Validation F1: 0.9911
Epoch 4/5, Train Loss: 0.0334, Validation F1: 1.0000
Epoch 5/5, Train Loss: 0.0233, Validation F1: 0.9946
Average F1 across folds: 0.9440


In [9]:
ruta = Path(os.getcwd()).parent / "models" / "saved_roberta_model"

In [10]:
utils.save_model_and_tokenizer(model, tokenizer, ruta)

Model and tokenizer have been saved to c:\Users\DELLPHOTO\Desktop\Fran\depression_detection\models\saved_roberta_model


# Inferencia

In [19]:
# Ejemplo de uso

model_path = ruta
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.to(device)  # Asegúrate de que el modelo esté en el mismo dispositivo (CPU o GPU)

text = "Yesterday i went to the movies with my friends"  # Texto de ejemplo para clasificación
probabilities, predicted_class = utils.make_prediction(text, model, tokenizer, device)
print(f"Probabilidades: {probabilities}")
print(f"Clase Predicha: {predicted_class}")